In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir("/Parastor300s_G30S/sunxh/MTB/v_snp2")

isolate_infor = pd.read_excel("/Parastor300s_G30S/sunxh/MTB/v_snp2/isolate_infor.xlsx")
all_snp=pd.read_table("all.vsnp.annotated")
mix_isolate_infor = pd.read_table("mix_isolate",header = None)
names = ['patient_id','stage','mix_infor']
mix_isolate_infor.columns = names
mix_isolate_infor.patient_id = mix_isolate_infor.patient_id.astype(int)
mix_isolate_infor.stage = mix_isolate_infor.stage.astype(int)
patient_infor = pd.read_excel("/Parastor300s_G30S/sunxh/MTB/v_snp2/population_infor.xlsx", skiprows = 10)

#patient_data2 = pd.DataFrame(patient_data_1bp)
patient_data2 = pd.read_csv("data1bp_depth.csv")
patient_data2.patient_id = patient_data2.patient_id.astype(int)
patient_data2.stage = patient_data2.stage.astype(int)
all_snp2 = pd.merge(all_snp,patient_data2,left_on=['patient_id','stage'],right_on=['patient_id','stage'], how='left')
all_snp2["max_depth_cutoff"] = all_snp2.mean_depth + 3*np.sqrt(all_snp2.mean_depth)
all_snp2["min_depth_cutoff"] = all_snp2.mean_depth - 6*np.sqrt(all_snp2.mean_depth)
all_snp_filtered = all_snp2[(all_snp2.coverage <= all_snp2.max_depth_cutoff) & (all_snp2.coverage >= all_snp2.min_depth_cutoff)]
all_snp_filtered2 = pd.merge(all_snp_filtered,mix_isolate_infor,left_on=['patient_id','stage'],right_on=['patient_id','stage'], how='left')
all_snp_filtered2 = all_snp_filtered2[all_snp_filtered2.mix_infor=="No"]




TIMEPOINT_DATA2 = {'PATIENT_ID': [], 
                  'TIME': [], 
                  'MEAN_FREQUENCY': [], 
                  'LOG_MEAN_FREQUENCY': [],
                  'MEDIAN_FREQUENCY': [],
                  'VARIANCE_FREQUENCY': [],
                  'vSNP_COUNT': [],
                  'DEPTH': [],
                  'Phenotype': []
                  }


for _patient in all_snp_filtered2.patient_id.unique():
    _timepoints = list(set(all_snp_filtered2.stage[all_snp_filtered2.patient_id==_patient]))
    for _timepoint in _timepoints:
        _frequencies = list(all_snp_filtered2.frequency[(all_snp_filtered2.stage==_timepoint)&(all_snp_filtered2.patient_id==_patient)])
        _depth = np.mean(list(all_snp_filtered2.mean_depth[(all_snp_filtered2.stage==_timepoint)&(all_snp_filtered2.patient_id==_patient)]))
        _mean_frequency = np.mean(_frequencies)
        _median_frequency = np.median(_frequencies)
        _variance = np.var(_frequencies)
        _effective = int(_patient in [41, 42, 43, 44,45,46,47,48,49,50])
        
        
        TIMEPOINT_DATA2['PATIENT_ID'].append(_patient)
        TIMEPOINT_DATA2['TIME'].append(_timepoint)
        TIMEPOINT_DATA2['MEAN_FREQUENCY'].append(_mean_frequency)
        TIMEPOINT_DATA2['LOG_MEAN_FREQUENCY'].append(np.log(_mean_frequency))
        TIMEPOINT_DATA2['MEDIAN_FREQUENCY'].append(_median_frequency)
        TIMEPOINT_DATA2['VARIANCE_FREQUENCY'].append(_variance)
        TIMEPOINT_DATA2['vSNP_COUNT'].append(len(_frequencies))
        TIMEPOINT_DATA2['DEPTH'].append(_depth)
        TIMEPOINT_DATA2['Phenotype'].append(_effective)


TIMEPOINT_DF_all_snp = pd.DataFrame(TIMEPOINT_DATA2)

all_snp_filtered3 = pd.merge(all_snp_filtered2,patient_infor,left_on=['patient_id'],right_on=['Case'], how='left')
all_snp_filtered4 = pd.merge(all_snp_filtered3,isolate_infor,left_on=['patient_id','stage'],right_on=['Case','Isolate'], how='left')


In [4]:
all_snp_filtered2.frequency.max()


1.0

In [ ]:
RECURRENT_vSNP = {}
for _patient in all_snp_filtered2.patient_id.unique():
    RECURRENT_vSNP[_patient] = {}
    locus = sorted(list(set(all_snp_filtered2.locus[all_snp_filtered2.patient_id==_patient])))
    month = sorted(list(set(all_snp_filtered2.stage[all_snp_filtered2.patient_id==_patient])))
    
    RECURRENT_vSNP[_patient]["chrom"] = []
    RECURRENT_vSNP[_patient]["gene_name"] = []
    RECURRENT_vSNP[_patient]["postion"] = locus
    RECURRENT_vSNP[_patient]["ref_allele"] = []
    RECURRENT_vSNP[_patient]["alt_allele"] = []
    RECURRENT_vSNP[_patient]["codon_change"] = []
    RECURRENT_vSNP[_patient]["codon_number"] = []
    RECURRENT_vSNP[_patient]["anocid"] = []
    RECURRENT_vSNP[_patient]["anocid_type"] = []
    
   
    
    for j in range(len(locus)):
        RECURRENT_vSNP[_patient]["ref_allele"].append(all_snp_filtered2.ref_base[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.locus==locus[j])].values[0])
        RECURRENT_vSNP[_patient]["alt_allele"].append(all_snp_filtered2.alt_base[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.locus==locus[j])].values[0])
        RECURRENT_vSNP[_patient]["chrom"].append("NC_000962.3") 
        RECURRENT_vSNP[_patient]["gene_name"].append(all_snp_filtered2.gene_name[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.locus==locus[j])].values[0])
        RECURRENT_vSNP[_patient]["codon_change"].append(all_snp_filtered2.condon_change[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.locus==locus[j])].values[0])
        RECURRENT_vSNP[_patient]["codon_number"].append(all_snp_filtered2.codon_number[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.locus==locus[j])].values[0])
        RECURRENT_vSNP[_patient]["anocid"].append(all_snp_filtered2.anocid[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.locus==locus[j])].values[0])
        RECURRENT_vSNP[_patient]["anocid_type"].append(all_snp_filtered2.anocid_type[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.locus==locus[j])].values[0])
        
        
         
        
    for i in range(len(month)):
        RECURRENT_vSNP[_patient][str(_patient) + "_m" + str(month[i])] = []
        for j in range(len(locus)):
            if all_snp_filtered2.frequency[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.stage==month[i])&(all_snp_filtered2.locus==locus[j])].values.size == 0:
                RECURRENT_vSNP[_patient][str(_patient) + "_m" + str(month[i])].append(0)
            else:
                RECURRENT_vSNP[_patient][str(_patient) + "_m" + str(month[i])].append(all_snp_filtered2.frequency[(all_snp_filtered2.patient_id==_patient)&(all_snp_filtered2.stage==month[i])&(all_snp_filtered2.locus==locus[j])].values[0])
            
        

os.chdir("/Parastor300s_G30S/sunxh/MTB/v_snp2")
for _patient in sorted(list(set(all_snp_filtered2.patient_id))):
    #print (pd.DataFrame(RECURRENT_vSNP[_patient]).T)
    pd.DataFrame(RECURRENT_vSNP[_patient]).T.to_csv("all_change_20220620.csv",mode='a')
    